In [ ]:
import pandas as pd
import numpy as np
import regex
from pathlib import Path

In [ ]:
HRSA_raw = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\hrsa\HRSA - 2021 - Cleaned.csv')

In [ ]:
# This dataset was created 5-27-2021 by HRSA
# tx_center Count - 254
print(HRSA_raw['OTC Name'].nunique())
# State Count - 48
print(HRSA_raw['Common State Name'].nunique())
# Cities Count - 137
print(HRSA_raw['OTC City'].nunique())
print(HRSA_raw.info())

In [ ]:
# Master list of OTC 2021
otc_2021 = HRSA_raw[['OTC Name', 'OTC Address', 'OTC City', 'Common State Name', 'Common State Abbreviation', 'OTC ZIP', 'OTC Longitude', 'OTC Latitude', 'OTC Code']]
otc_2021 = otc_2021.drop_duplicates().reset_index(drop=True)

#OPO master list
opo = HRSA_raw[['OPO Name', 'OPO Address', 'OPO City', 'OPO State', 'OPO ZIP', 'OPO Telephone #', 'OPO Provider #']]
opo = opo.drop_duplicates().reset_index(drop=True)

# Organs transplanted from each OTC
otc_service = HRSA_raw[['OTC Code', 'OTC Service List', 'Organ Transplantation Center Service Type Description']]

# Regions Lists
regions = HRSA_raw[['Common Region Name', 'Common State Name', 'Common Region Code', 'OPO Provider #', 'OTC Code']]
regions = regions.drop_duplicates().reset_index(drop=True)

opo_regions = regions[['Common Region Code', 'OPO Provider #']]
opo_regions = opo_regions.drop_duplicates().reset_index(drop=True)

otc_regions = regions[['Common Region Code', 'OTC Code']]
otc_regions = otc_regions.drop_duplicates().reset_index(drop=True)

regions = regions.drop(columns=['OPO Provider #', 'OTC Code'])
regions = regions.drop_duplicates().reset_index(drop=True)

# OPO key = OPO ZIP
# OTC key = OTC ZIP
hrsa_census_keys = HRSA_raw[['Common State FIPS Code', 'Common State County FIPS Code', 'Common County Name', 'Common City Name with State Abbreviation', 'Common Congressional District Name', 'Common State FIPS Code Congressional District Number', 'Common Postal Code']]
hrsa_census_keys = hrsa_census_keys.drop_duplicates()

In [ ]:
srtr_raw = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\otc-2009-raw.csv')

In [ ]:
srtr_raw['OTC Address'] = srtr_raw['OTC Address'].astype(object)
srtr_raw['Common State Name'] = srtr_raw['Common State Name'].astype(object)
srtr_raw['OTC ZIP'] = srtr_raw['OTC ZIP'].astype(object)
otc_2021['OTC ZIP'] = otc_2021['OTC ZIP'].astype(object)

In [ ]:
srtr_raw.merge(otc_2021, how='inner', on='OTC Code')

In [ ]:
hospitals = [srtr_raw, otc_2021]
pd.concat(hospitals, join='inner', keys='OTC Code')

In [ ]:
srtr_pending = srtr_raw.loc[srtr_raw['OTC Code'].isnull()].reset_index(drop=True)
srtr_finished = srtr_raw.loc[srtr_raw['OTC Code'].notnull()].reset_index(drop=True)

In [ ]:
srtr_pending_manual = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\srtr_pending_manual.csv')

In [ ]:
srtr_pending_2 = srtr_pending_manual.loc[srtr_pending_manual['OTC Code'].isnull()].reset_index(drop=True)
srtr_finished_2 = srtr_pending_manual.loc[srtr_pending_manual['OTC Code'].notnull()].reset_index(drop=True)

In [ ]:
finisheds = [srtr_finished, srtr_finished_2]

In [ ]:
srtr_pending_2.head(1)
# Drop Common State Name
srtr_pending_2 = srtr_pending_2.drop(columns=['Common State Name'])

In [ ]:
srtr_pending_2.head(1)

In [ ]:
us_hospitals_full = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\us_hospital_locations.csv')

In [ ]:
us_hospitals = us_hospitals_full[['NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'LONGITUDE', 'LATITUDE']]

In [ ]:
# Don't really need an OTC Code for 2009
srtr_finished_2 = srtr_pending_2.drop(columns=['OTC Code'])

In [ ]:
srtr_pending_2['OTC Name'] = srtr_pending_2['OTC Name'].str.upper()

In [ ]:
us_hospitals.NAME = us_hospitals.NAME.str.upper()

In [ ]:
us_hospitals = us_hospitals.rename(columns={'NAME' : 'OTC Name', 'ADDRESS' : 'OTC Address', 'CITY' : 'OTC City', 'STATE' : 'Common State Abbreviation', 'ZIP' : 'OTC ZIP', 'LONGITUDE' : 'OTC Longtitude', 'LATITUDE' : 'OTC Latitude'})

In [ ]:
srtr_pending_2['OTC City'] = srtr_pending_2['OTC City'].str.upper()
srtr_pending_2['key'] = srtr_pending_2['OTC Name'] + srtr_pending_2['OTC City'] + srtr_pending_2['Common State Abbreviation']
us_hospitals['key'] = us_hospitals['OTC Name'] + us_hospitals['OTC City'] + us_hospitals['Common State Abbreviation']

In [ ]:
pd.set_option('display.max_row', 10)


In [ ]:
srtr_pending_2.head(1)

In [ ]:
us_hospitals.head(1)

In [ ]:
test = srtr_pending_2[['key']]

In [ ]:
test2 = test.merge(us_hospitals, how='left', on='key')
# Drop key, drop/fix OTC name column

In [ ]:
test2.info()

In [ ]:
pending3 = test2.loc[test2['OTC ZIP'].isnull()].reset_index()

In [ ]:
pending3

In [ ]:
%who

In [ ]:
still_looking = 